## Basic Imports

In [ ]:
import cufflinks 
import numpy as np
import pandas as pd
from pylab import plt
import pyfolio as pf
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment',None)
cufflinks.set_config_file(offline=True)
%config InlineBackend.figure_format='svg'

import warnings
warnings.filterwarnings('ignore')

import plotly.graph_objs as go #candelistic chart

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## Financial Data Class

In [ ]:
data_file='modified_nifty_5min.csv'
date='2021-03-02'

In [ ]:
class FiancialData:
    def __init__(self,data_file):
        self.data_file=data_file
        self.get_data()
        self.prepare_data()
    def get_data(self):
        self.raw=pd.read_csv(self.data_file,index_col=0,parse_dates=True).dropna()
    def prepare_data(self):
        self.df=self.raw.copy()
        self.df['per_candle_return']=np.log(self.df['open']/self.df['open'].shift(1))
    def plot_data(self):
        self.df.iloc[:,1:5].iplot()
    def plot_candlestick_chart(self,date):
        self.date=date
        self.chart_df = [go.Candlestick(x=self.df['Time'][self.date], 
                             open =self.df['open'][self.date], 
                             high =self.df['high'][self.date], 
                             low = self.df['low'][self.date],
                             close =self.df['close'][self.date])]
        # Load chart df
        self.fig = go.Figure(data=self.chart_df)
        
        # Update chart layout
        self.fig.update_layout(xaxis_rangeslider_visible=False, xaxis_showticklabels=True,
                               yaxis_showticklabels=True)
        # Plot chart
        self.fig.show()
        
        
        

In [ ]:
fd=FiancialData(data_file)

In [ ]:
fd.plot_data()

In [ ]:
fd.plot_candlestick_chart(date)

sma_20=20
sma_200=200

In [ ]:
class Backtesting_Strategy(FiancialData):
    sma_20=20
    sma_200=200
    def __init__(self,data_file):
        super().__init__(data_file)
        self.SMA_cross_over()
        self.Colour_column()
        self.percentage_candle()
        self.Condition_filters()
        self.Time_filters()
        self.core_columns()
        self.print_results()    
        
        
        
    #columns creation   
    def SMA_cross_over(self):
        #creating a columns for moving average 20 and 200days
        self.df['sma_20']=self.df['close'].rolling(window=Backtesting_Strategy.sma_20,center=False).mean()
        self.df['sma_200']=self.df['close'].rolling(window=Backtesting_Strategy.sma_200,center=False).mean()
    
    def Colour_column(self):
        #colour code just for reference can be ignored later
        self.df['colour']=np.where(self.df['close']>self.df['open'],'Green','Red')
        #if any nan values are available we will drop it
        self.df.dropna(inplace=True)
        
    def percentage_candle(self):
        #To know the percentage of body created in candle(We need >80% body formation)
        self.df['candle']=(abs(self.df['open']-self.df['close'])/abs(self.df['high']-self.df['low']))*100
    
    def Condition_filters(self):
        self.candle_body=75 #85% body is desired
        self.candle_size_points=40 #nifty below 40points,banknifty 100 and differs accordingly
        self.distance_sma20_to_candle=20 #distance b/w sma20 and candle should be minimum
        self.low_wig=6 #not mandatory
        self.high_wig=5 #not mandatory
        
        #buy conditions
        self.df['condition1']= np.where((self.df['sma_20']>self.df['sma_200'])& #crossover-buy condition
                            (self.df['low']>self.df['sma_200'])&  #improving candle condition for better fit
                            (self.df['sma_20']>self.df['sma_20'].shift(4)) & #challenges of curve sma
                            (self.df['sma_20'].shift(4)>self.df['sma_20'].shift(8))& #curve sma
                            (self.df['candle']>self.candle_body)& #candle body %
                            (self.df['high']-self.df['low']<=self.candle_size_points)& #sometimes candles are too big so to restrict
                            (abs(self.df['low']-self.df['sma_20'])<self.distance_sma20_to_candle)& #to reduced the distance b/w candle and sma20
                            (self.df['colour']=='Green')& #buy means green
                            (self.df['close']>self.df['sma_20'])& #other conditions
                            ((self.df['low']>=self.df['sma_20'])|
                             (self.df['low']==self.df['open'])| #or conditions for simillar strategy        
                             abs(self.df['low']-self.df['open']<self.low_wig)|
                             (self.df['high']==self.df['close'])|
                             ((self.df['high']-self.df['close'])<= self.high_wig)),1,0)
        
        #sell conditions
        self.df['condition1']= np.where((self.df['sma_20']<self.df['sma_200'])& #simillar as above but for sell
                            (self.df['high']<self.df['sma_200'])&
                            (self.df['sma_20']<self.df['sma_20'].shift(4)) &
                            (self.df['sma_20'].shift(4)<self.df['sma_20'].shift(8))&
                            (self.df['candle']>self.candle_body)&
                            (self.df['high']-self.df['low']<=self.candle_size_points)& 
                            (abs(self.df['high']-self.df['sma_20'])<self.distance_sma20_to_candle)&
                            (self.df['colour']=='Red')&
                            (self.df['close']<self.df['sma_20'])&
                            ((self.df['high']<=self.df['sma_20'])|
                             (self.df['high']==self.df['open'])|
                             (self.df['low']==self.df['close'])|
                             (abs(self.df['low']-self.df['close'])<=self.low_wig)|
                             abs(self.df['high']-self.df['open']< self.high_wig)),-1,self.df['condition1'])
    def Time_filters(self):
        self.df['condition2']=np.where((self.df['Time']>'09:15:00') & (self.df['Time'] <'10:55:00')
                            | (self.df['Time']>'12:25:00') & (self.df['Time']<'12:55:00')
                            | (self.df['Time']>'13:45:00') & (self.df['Time'] <'15:00:00'),
                           np.where(self.df['condition1']==1,1,
                                    np.where( self.df['condition1']==-1,-1,0)),0)
              
        #shifting the condition2 column by one
        self.df['condition2']=self.df['condition2'].shift(1)
        
    def core_columns(self):
        self.df['signal']=""
        self.df['position']=0
        self.df['trade_price']=0
        self.df['trade_return']=0
        
        for i in range(0,len(self.df)):
            #if there is an open position carry the position and the entery price
            if self.df.iloc[i-1].position !=0:
                print(i,str(self.df.index[i]),'carry position')
                self.df.position.iloc[i]=self.df.iloc[i-1].position
                self.df.trade_price.iloc[i]=self.df.iloc[i-1].trade_price
            
            #Entery long
            #if after getting morubuzu conformation the neext candle cross the high of morubuzu we go long
            if self.df.iloc[i].high >self.df.iloc[i-1].high and self.df['condition2'].iloc[i]==1 and self.df.iloc[i].position==0:
                print(i,str(self.df.index[i]),'Entery long,price',self.df.iloc[i].close)
                self.df.position.iloc[i]=1
                #trade price id morubuzu high
                self.df.trade_price.iloc[i]=self.df.iloc[i-1].high 
                self.df.signal.iloc[i]='Entery long'
                stop_loss=self.df.low.iloc[i-1] #like guide suggested price previous candle low
                take_profit=self.df.open.iloc[i+4] #price open of 5th candle/completion of 4th candle
            
            #check take profit and stop loss for long position
            elif self.df.iloc[i].position==1:
                #if current stock price>tp or current price <sl:
                #squareoff here
                if self.df.iloc[i].open == take_profit:
                    print(i,str(self.df.index[i]),'Take profit long',self.df.iloc[i].open)
                    self.df.position.iloc[i]=0
                    self.df.trade_return.iloc[i]=self.df.iloc[i].open/self.df.iloc[i].trade_price-1
                    self.df.trade_price.iloc[i]=self.df.iloc[i].open
                    self.df.signal.iloc[i]='TP long'
                elif self.df.iloc[i].low <= stop_loss: #or self.df.iloc[i].low < stop_loss: #bug should correct
                    print(i,str(self.df.index[i]),'Stoploss long,',self.df.iloc[i].open)
                    self.df.position.iloc[i]=0
                    self.df.trade_return.iloc[i]=self.df.iloc[i].open/self.df.iloc[i].trade_price-1
                    self.df.trade_price.iloc[i]=self.df.iloc[i].open
                    self.df.signal.iloc[i]='SL long'
                    
            #Entery short
            #if after getting morubuzu conformation the neext candle cross the low of morubuzu we go short
            elif self.df.iloc[i].high <self.df.iloc[i-1].low and self.df['condition2'].iloc[i]==-1 and self.df.iloc[i].position==0:
                print(i,str(self.df.index[i]),'Entery short,price',self.df.iloc[i].close) #doubt in close
                self.df.position.iloc[i]=-1
                #trade price is morubuzu low
                self.df.trade_price.iloc[i]=self.df.iloc[i-1].low
                self.df.signal.iloc[i]='Entery short'
                stop_loss=self.df.high.iloc[i-1] # price previous candle high
                take_profit=self.df.open.iloc[i+4] #price open of 5th candle/completion of 4th candle
                
            #check take profit and stop loss for short position
            elif self.df.iloc[i].position ==-1:
                #if current stock price<tp or current price >sl:
                #squareoff here
                if self.df.iloc[i].open == take_profit:
                    print(i,str(self.df.index[i]),'Take profit short',self.df.iloc[i].open)
                    self.df.position.iloc[i]=0
                    self.df.trade_return.iloc[i]=self.df.iloc[i].trade_price/self.df.iloc[i].open -1
                    self.df.trade_price.iloc[i]=self.df.iloc[i].open
                    self.df.signal.iloc[i]='TP short'
                                    
                elif self.df.iloc[i].high >= stop_loss: #or self.df.iloc[i].high > stop_loss:
                    print(i,str(self.df.index[i]),'Stoploss short,',self.df.iloc[i].open)
                    self.df.position.iloc[i]=0
                    self.df.trade_return.iloc[i]=self.df.iloc[i].trade_price/self.df.iloc[i].open -1
                    self.df.trade_price.iloc[i]=self.df.iloc[i].open
                    self.df.signal.iloc[i]='SL short'
            
    def print_results(self):
        # Compute the strategy returns
        #self.df['strategy_return'] = self.df['per_candle_return'] * self.df['position'].shift(1)
        
        #trade return
        print('Total Trade returns =',self.df['trade_return'].sum() *100,'%')
        #print('The strategy returns =',self.df['strategy_return'].cumsum()[-1])
        print('If buy and holded the same =',self.df['per_candle_return'].cumsum()[-1]*100,'%')
        return self.df['trade_return'].sum() *100
    
    def summary(self):
        
        self.df['Strategy_Return']=self.df['trade_return'].expanding().sum()
        self.df['Wins']=np.where(self.df['trade_return'] > 0,1,0)
        self.df['Losses']=np.where(self.df['trade_return']<0,1,0)
        
        ## Daywise Performance
        self.d_perform = {}
        self.d_perform['TotalWins']=self.df['Wins'].sum()
        self.d_perform['TotalLosses']=self.df['Losses'].sum()
        self.d_perform['TotalTrades']=self.d_perform['TotalWins']+self.d_perform['TotalLosses']
        self.d_perform['HitRatio']=round(self.d_perform['TotalWins']/self.d_perform['TotalTrades'],2)
        self.d_perform['SharpeRatio'] = self.df["trade_return"].mean() / self.df['trade_return'].std() * (252**.5)
        self.d_perform['CAGR'] = (1+self.df['Strategy_Return']).iloc[-1]**(365.25/len(self.df)) -1
        self.daywise_performance =pd.Series(self.d_perform)
        print(self.daywise_performance)
        #daywise_performance
        
        self.d_tp = {}
        self.d_tp.update(self.df[["Wins","Losses"]].sum().rename({'Wins':'TotalWins','Losses':'TotalLosses'}).to_dict())
        self.d_tp['TotalTrades'] = self.d_tp["TotalWins"] + self.d_tp["TotalLosses"]
        self.d_tp['HitRatio'] =  np.round(self.d_tp["TotalWins"] / self.d_tp['TotalTrades'],4)
        self.d_tp['AvgWinRet'] = np.round(self.df[self.df.Wins==1].trade_return.mean(),4)
        self.d_tp['AvgLossRet'] = np.round(self.df[self.df.Losses==1].trade_return.mean(),4)
        self.d_tp['WinByLossRet'] = np.round(abs(self.d_tp['AvgWinRet']/self.d_tp['AvgLossRet']),2)
        self.d_tp['RetVar'] = np.round(self.df.trade_return.std(),4)
        self._sum = self.df.groupby("Wins").trade_return.sum()
        self.d_tp['NormHitRatio'] = np.round(self._sum[1]/self._sum.abs().sum(),4)
        #d_tp['OptimalTradeSize'] = self.kelly(p = d_tp['HitRatio'], b = d_tp['WinByLossRet'])
        self.tradewise_performance = pd.Series(self.d_tp)
        return self.tradewise_performance        
        
    def plot_analysis(self):
        
        #self.df[['per_candle_return', 'strategy_return']].cumsum().plot(grid=True, figsize=(9,5))
        self.df['trade_return'].iplot(title='Trade Returns',yTitle='Returns')
        pf.create_simple_tear_sheet(self.df['trade_return'])
          

In [ ]:
%%time
data=Backtesting_Strategy(data_file)

In [ ]:
data.plot_analysis()

In [ ]:
data.summary()

In [ ]:
data.print_results()